In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler

In [ ]:
def print_results_crossvalidation(func, X_test, y_test):
  
  std_best_score = func.cv_results_["std_test_score"][func.best_index_]
  print(f"Best parameters: {func.best_params_}")
  print(f"Mean CV score: {func.best_score_:}")
  print(f"Standard deviation of CV score: {std_best_score:}")
  print("Test Score:".format(func.score(X_test, y_test)))

def report(y_true, y_pred):
    
  class_report = metrics.classification_report(y_true, y_pred)
  print(class_report)
  conf_matrix = confusion_matrix(y_true, y_pred, normalize = "all")
  conf_matrix = pd.DataFrame(conf_matrix, ["Class 0", "Class 1", " Class 2", "Class 3", " Class 4"],  ["Class 0", "Class 1", " Class 2", "Class 3", " Class 4"])
  sns.heatmap(conf_matrix, annot = True).set(xlabel = "Assigned Class", ylabel = "True Class", title = "Confusion Matrix")
     

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
plt.rcParams['font.size'] = 10

In [ ]:
# Load data
df = pd.read_csv("GroupProjectDataSet.csv", sep=',')
print('Shape of data frame:', df.shape)
df.head(10)

In [ ]:
# Filling missing values for variables where appropriate

df["PoolQC"] = df["PoolQC"].fillna(value = "None")
df["MiscFeature"] = df["MiscFeature"].fillna(value = "None")
df["Alley"] = df["Alley"].fillna(value = "None")
df["Fence"] = df["Fence"].fillna(value = "None")
df["FireplaceQu"] = df["FireplaceQu"].fillna(value = "None")
df["GarageCond"] = df["GarageCond"].fillna(value = "None")
df["GarageType"] = df["GarageType"].fillna(value = "None")
df["GarageFinish"] = df["GarageFinish"].fillna(value = "None")
df["GarageQual"] = df["GarageQual"].fillna(value = "None")
df["BsmtFinType2"] = df["BsmtFinType2"].fillna(value = "None")
df["BsmtExposure"] = df["BsmtExposure"].fillna(value = "None")
df["BsmtQual"] = df["BsmtQual"].fillna(value = "None")
df["BsmtCond"] = df["BsmtCond"].fillna(value = "None")
df["BsmtFinType1"] = df["BsmtFinType1"].fillna(value = "None")

In [ ]:
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]
missing.plot.bar()

In [ ]:
# Percentage of missing values for the variables

percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Share'])
missing_data.head(5)

In [ ]:
#####Dealing with Categorical Features (Encoding Categorical Variables) / Splitting Into X and y

# Numerical variables that should be handled as categorical variables
df = df.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"}})
df = df.replace({"MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}})

In [ ]:
# Assign response to y
y = df.iloc[:, -1]

# Factorize categorical values, assign output to X
# create (multiple) dummy variables for a categorical variable
# panda way
X = pd.get_dummies(df.iloc[:, :-1])
X.head()

In [ ]:
#Ab hier übernehmen

In [ ]:
## Feature Selection##
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Compute the correlation matrix
corr = df.corr()

# Plotting Heatmap
plt.figure(figsize = (10,6))
sns.heatmap(corr, annot = True)

In [ ]:
# Compute the correlation matrix
corr_matrix = df.corr()

# Sort the correlations with respect to 'Class'
corr_with_class = corr_matrix['Class'].sort_values(ascending=False)

# Print the correlations
print(corr_with_class)

# Select the top 10 features with the highest correlation
top_features = corr_with_class.nlargest(10).index

# Print the top features
print(top_features)

In [ ]:
##VIF## 

# load the data and select features
X = df[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
        'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]

# add a constant to X for the intercept
X = sm.add_constant(X)

# calculate VIF for each feature
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

# print the VIF table
print(vif)


In [ ]:
#######KNN#######

In [ ]:
####Partitioning of the Data Set Into Train and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42, 
                                                    stratify=y)

In [ ]:
#KNN-Model##
train_df = pd.DataFrame(data=X_train, columns=X.columns)
train_df['Class'] = y_train

test_df = pd.DataFrame(data=X_test, columns=X.columns)
test_df['Class'] = y_test

X_train = train_df[['Class', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
y_train = train_df['Class']

# Initialize the KNN classifier with k=20
knn = KNeighborsClassifier(n_neighbors=20)

# Fit the KNN model on the training data
knn.fit(X_train, y_train)

# Use the trained KNN model to make predictions on the test data
X_test = test_df[['Class', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
y_test = test_df['Class']
y_pred = knn.predict(X_test)

# Compute the accuracy of the KNN model on the test data
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Compute the classification report for the KNN model on the test data
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
# The model correctly classified 75% of the instances in the test set.
# The report indicates that the model performs well in predicting class 1, 
# with a precision of 0.82 and a recall of 0.90, indicating that it correctly predicted a high proportion of 
# the instances belonging to this class. However, the model performed less well in predicting classes 0 and 3, 
# with a precision and recall of around 0.5. The model had moderate performance for class 2, with a precision 
#and recall of around 0.6, and poor performance for class 4, with a precision of 0.5 and a recall of 0.38.

In [ ]:
# Define a range of values for k
k_range = range(1, 80)

# Create an empty list to store the cross-validation scores
cv_scores = []

# Perform k-fold cross-validation for each value of k
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(np.mean(scores))

# Plot the cross-validation scores as a function of k
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(k_range, cv_scores)
plt.xlabel('Value of k for KNN')
plt.ylabel('Cross-Validation Accuracy')
plt.show()

# Select the best value of k
best_k = np.argmax(cv_scores) + 1
print("The best value of k is:", best_k)


In [ ]:
# we can set k=20 in our KNN model to get the best possible performance on this particular dataset.

In [ ]:
###KNN-SMOTE####
# Create SMOTE object
smote = SMOTE(random_state=42)

# Fit SMOTE on the training data
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Print the number of samples before and after SMOTE
print("Number of samples before SMOTE:", len(X_train))
print("Number of samples after SMOTE:", len(X_train_smote))

# Train a KNN classifier on the oversampled data
knn_smote = KNeighborsClassifier(n_neighbors=20)
knn_smote.fit(X_train_smote, y_train_smote)

# Predict the test set using the KNN classifier trained on the oversampled data
y_pred_smote = knn_smote.predict(X_test)

# Compute the accuracy of the KNN model with SMOTE on the test data
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print("Accuracy with SMOTE:", accuracy_smote)

# Compute the classification report for the KNN model with SMOTE on the test data
report_smote = classification_report(y_test, y_pred_smote)

# Print the classification report
print(report_smote)




In [ ]:
# SMOTE is used to address class imbalance by creating synthetic samples.
# However, in this case, after oversampling with SMOTE, the performance of the KNN classifier on the test set 
# actually got worse compared to the previous results without SMOTE. The accuracy dropped from 0.7534 to 0.621, 
# and the overall classification report shows lower precision and recall scores for some classes. 
#This could be due to overfitting on the training data or other factors

In [ ]:
##Class Imbalance using Random Oversample
# Initialize the random oversampler with random_state=0
ros = RandomOverSampler(random_state=0)

# Resample the training data
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Initialize the KNN classifier with k=20
knn = KNeighborsClassifier(n_neighbors=20)

# Fit the KNN model on the resampled training data
knn.fit(X_train_resampled, y_train_resampled)

# Use the trained KNN model to make predictions on the test data
y_pred = knn.predict(X_test)

# Compute the accuracy of the KNN model on the test data
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy with RandomOverSampler:", accuracy)

# Compute the classification report for the KNN model on the test data
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)


In [ ]:
#  RandomOverSampler technique to address the class imbalance in the training data. 
# The accuracy of the model after using Random Oversampling is 0.589. The drop in accuracy could be due to the 
#fact that the synthetic samples generated by Random Oversampling can add noise to the data, leading to overfitting. 

In [ ]:
# Define features and target variable
X = df[['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
y = df['Class']

# Create KNN object and run classifier
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X, y)

# Predict probabilities for each class
y_prob = knn.predict_proba(X)

# Compute ROC curve and ROC area for each class
fpr = {}
tpr = {}
roc_auc = {}
for i in range(len(set(y))):
    fpr[i], tpr[i], _ = roc_curve(y, y_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['red', 'green', 'blue', 'orange', 'purple']
for i, color in zip(range(len(set(y))), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
#####LDA#####

In [ ]:
# Feature-Matrix X und Zielvariable Y auswählen
X = df[['OverallQual', 'GrLivArea', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
Y = df['Class']

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

# LDA-Modell erstellen und anpassen
lda = LDA(solver='lsqr')
lda.fit(X_train, y_train)

# Vorhersagen auf Testdaten machen
y_pred = lda.predict(X_test)

# Leistungsmaße berechnen
print('default-rate: {0: .4f}'.format(np.sum(y_test)/len(y_test)))

report = classification_report(y_test, y_pred)
print(report)

# Konfusionsmatrix ausgeben
print(metrics.confusion_matrix(y_test, y_pred))


In [ ]:
# In this specific case, the default rate is 1.29, which means that the most frequent class in the test set is Class 1
# The F1-score is 0.79, which indicates that the model has good accuracy. 
# The classification report shows precision, recall, and F1-score for each class.

In [ ]:
###SMOTE###
# Feature-Matrix X und Zielvariable Y auswählen
X = df[['OverallQual', 'GrLivArea', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
Y = df['Class']

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Apply SMOTE to training data
sm = SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# LDA-Modell erstellen und anpassen
lda = LDA(solver='lsqr')
lda.fit(X_train_res, y_train_res)

# Vorhersagen auf Testdaten machen
y_pred = lda.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

# Konfusionsmatrix ausgeben
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
# Here, we have also tried to address the class imbalancing problem by applying SMOTE. 
# But this also led to a deterioration of the model.

In [ ]:
# Define features and target variable
X = df[['OverallQual', 'GrLivArea', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
y = df['Class']

# Create LDA object and run classifier
lda = LDA(solver='lsqr')
lda.fit(X, y)

# Predict probabilities for each class
y_prob = lda.predict_proba(X)

# Compute ROC curve and ROC area for each class
fpr = {}
tpr = {}
roc_auc = {}
for i in range(len(set(y))):
    fpr[i], tpr[i], _ = roc_curve(y, y_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['red', 'green', 'blue', 'orange', 'purple']
for i, color in zip(range(len(set(y))), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
####QDA
# Feature-Matrix X und Zielvariable Y auswählen
X = df[['OverallQual', 'GrLivArea', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
Y = df['Class']

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# QDA-Modell erstellen und anpassen
qda = QDA()
qda.fit(X_train, y_train)

# Vorhersagen auf Testdaten machen
y_pred = qda.predict(X_test)

# Konfusionsmatrix ausgeben
print(metrics.confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))


In [ ]:
# The model achieves an accuracy of 0.82, and the precision and recall vary across the classes, 
# with the highest precision and recall achieved for class 1.0.

In [ ]:
# Define features and target variable
X = df[['OverallQual', 'GrLivArea', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'FullBath', 'YearRemodAdd']]
y = df['Class']

# Create QDA object and run classifier
qda = QDA()
qda.fit(X, y)

# Predict probabilities for each class
y_prob = qda.predict_proba(X)

# Compute ROC curve and ROC area for each class
fpr = {}
tpr = {}
roc_auc = {}
for i in range(len(set(y))):
    fpr[i], tpr[i], _ = roc_curve(y, y_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['red', 'green', 'blue', 'orange', 'purple']
for i, color in zip(range(len(set(y))), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-Curve')
plt.legend(loc="lower right")
plt.show()
